In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.gridspec import GridSpec

from effmed.lib.matrix_model import effective_medium
from effmed.lib.supplemental import dB #, rotational_transform, find_cpe

from impdar.lib.ApresData.load_quadpol import load_quadpol_fujita

%matplotlib inline

In [ ]:
def get_poincare_sphere(dat):
    E0x = abs(dat.HH)
    E0y = abs(dat.HV)
    ϕHH = np.angle(dat.HH)
    ϕHV = np.angle(dat.HV)
    ϕ = ϕHH-ϕHV
    ϕ[ϕ<-np.pi] += 2.*np.pi
    ϕ[ϕ>np.pi] -= 2.*np.pi
    γ = np.arctan(E0y/E0x)

    S1 = E0x**2. - E0y**2.
    S2 = 2.*E0x*E0y*np.cos(ϕ)
    S3 = 2.*E0x*E0y*np.sin(ϕ)
    
    return S1,S2,S3

In [ ]:
%matplotlib widget 

plt.figure()

ax1 = plt.subplot(111,projection='3d')
ax1.plot([0,0],[0,0],[0,1.25],color='lightgrey')
ax1.plot([0,0],[0,-1.25],[0,0],color='lightgrey')
ax1.plot([0,1.25],[0,0],[0,0],color='lightgrey')

N = 100
ts = np.linspace(0,np.pi/2.,N)

ax1.plot(np.cos(ts),-np.sin(ts),np.zeros(N),color='darkgrey')
ax1.plot(np.zeros(N),-np.sin(ts),np.cos(ts),color='darkgrey')
ax1.plot(np.cos(ts),np.zeros(N),np.sin(ts),color='darkgrey')

gamma = np.linspace(0,.25*np.pi,100)
ay = np.sin(gamma)
ax = np.cos(gamma)
ϕ = np.linspace(0,.5*np.pi,100)

idx = 40
ax1.plot(2.*ax*ay*np.cos(ϕ[idx]),(ay**2.-ax**2.)*np.ones(100),2.*ax*ay*np.sin(ϕ[idx]),'k')
ax1.plot(2.*ax[idx]*ay[idx]*np.cos(ϕ),
         (ay[idx]**2.-ax[idx]**2.)*np.ones(100),
         2.*ax[idx]*ay[idx]*np.sin(ϕ),'k')
ax1.plot(2.*ax[idx]*ay[idx]*np.cos(ϕ[idx]),
         ay[idx]**2.-ax[idx]**2.,
         2.*ax[idx]*ay[idx]*np.sin(ϕ[idx]),'.',ms=15,mew=2,mec='k',mfc='w')

ax1.axis('off')
ax1.set_xlim(-.1,1.3)
ax1.set_ylim(-1.3,.1)
ax1.set_zlim(-.1,1.3)
ax1.axis('equal')

plt.savefig('Sphere_Notation.pdf')

In [ ]:
N = 100
ts = np.linspace(0,2*np.pi,N)
xx = np.cos(ts)
yy = np.sin(ts)
zz = np.zeros(N)

plt.figure()

ax2 = plt.subplot(111,projection='3d')
# draw sphere
u, v = np.mgrid[0:2*np.pi+.05:np.pi/20, 0:np.pi+.05:np.pi/20]
x = np.cos(u)*np.sin(v)
y = np.sin(u)*np.sin(v)
z = np.cos(v)
# alpha controls opacity
ax2.plot_wireframe(y, z, x, color="k", alpha=0.1)

plt.plot(xx,yy,zz,c='grey')
plt.plot([1,-1,0,0],[0,0,-1,1],[0,0,0,0],'k.')

plt.axis('equal')
plt.axis('off')

plt.savefig('PoincareSphere.pdf')

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

em.epsr=3.15
chis = [.1,0.,0.9]
em.ice_properties(T=253.,epsr=3.15,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(chis[0]-chis[1])/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
layer_dz = 1.
zs = np.arange(.1,H+8.*layer_dz,layer_dz)
dzs = layer_dz*np.ones(len(zs))
chi1 = 0.1*np.ones(len(zs))
chi2 = 0.0*np.ones(len(zs))
chi3 = 0.9*np.ones(len(zs))
chis = np.transpose([chi1,chi2,chi3])
thetas = np.zeros(len(zs))
psis = np.zeros(len(zs))

em.solve(zs,dzs,thetas,psis,chis)

dat1 = load_quadpol_fujita(em)
dat1.rotational_transform(n_thetas=401)
Θs,Ds = np.meshgrid(dat1.thetas,dat1.range)
dat1.find_cpe(5e7,rad_start=.1*np.pi,rad_end=.6*np.pi)
dat1.cpe[dat1.cpe==.1*np.pi] = .5*np.pi

dat1.coherence2d(delta_theta=.008,delta_range=1.1)
#dat1.phase_gradient2d()

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.333,.333,.333]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

H = 1000
zs = np.arange(10,H)
thetas=0.
psis = 0.001
gammas = np.array([1,.2])
em.solve(zs,H,thetas,psis,chis,gammas=gammas)

dat2 = load_quadpol_fujita(em)
dat2.rotational_transform(n_thetas=401)
dat2.find_cpe()
dat2.coherence2d(.15,layer_dz+.1)

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.35,.15,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(chis[0]-chis[1])/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
layer_dz = 1.
zs = np.arange(.1,H+8.*layer_dz,layer_dz)
thetas=0.
psis = 0.
gammas = np.array([1.,.2])
em.solve(zs,H,thetas,psis,chis,gammas=gammas)

dat3 = load_quadpol_fujita(em)
dat3.rotational_transform(n_thetas=401)
dat3.find_cpe()
dat3.coherence2d(.15,2.)

In [ ]:
u, v = np.mgrid[0:2*np.pi+.01:np.pi/10, 0:np.pi:25j]
x = np.cos(u)*np.sin(v)
y = np.sin(u)*np.sin(v)
z = np.cos(v)

plt.figure(figsize=(12,4))
ax1 = plt.subplot(131,projection='3d')
ax1.plot_wireframe(y, z, x, color="k", alpha=0.025)
ax2 = plt.subplot(132,projection='3d')
ax2.plot_wireframe(y, z, x, color="k", alpha=0.025)
ax3 = plt.subplot(133,projection='3d')
ax3.plot_wireframe(y, z, x, color="k", alpha=0.025)

dats = [dat1,dat2,dat2]
axs = [ax1,ax2,ax3]

dat = dat1
S1, S2, S3 = get_poincare_sphere(dat)
for i in range(0,len(S1[0])//2+2,20): #range(1,len(S1[0])//2+2,20):
    #i = len(S1[0]) - i - 200
    ax1.plot(S2[:,i],-S1[:,i],S3[:,i],lw=2,c=cm.twilight_shifted(i/200))
#ax1.plot(S1[0,200],S2[0,200],S3[0,200],'.',c=cm.twilight_shifted(.5))
i = 180
#ax1.plot(S1[0,i],S2[0,i],S3[0,i],'.',c=cm.twilight_shifted(1.))

dat = dat2
S1, S2, S3 = get_poincare_sphere(dat)
for i in range(0,len(S1[0]),20):
    ax2.plot(S2[0,i],-S1[0,i],S3[0,i],'.',ms=3.5,c=cm.twilight_shifted(i/400))
    
dat = dat3
S1, S2, S3 = get_poincare_sphere(dat)
for i in range(len(S1[0])-1,-1,-20):
    ax3.plot(S2[:,i],-S1[:,i],S3[:,i],lw=2,c=cm.twilight_shifted(i/400))
ax3.plot(S2[0,400],-S1[0,400],S3[0,400],'.',c=cm.twilight_shifted(.5))
ax3.plot(S2[0,0],-S1[0,0],S3[0,0],'.',c=cm.twilight_shifted(1.))

ax1.axis('equal')
ax1.axis('off')
ax2.axis('equal')
ax2.axis('off')
ax3.axis('equal')
ax3.axis('off')

plt.savefig('Sphere_Birefringence_And_Scattering.pdf')

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.35,.15,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(chis[0]-chis[1])/(2.*np.sqrt(em.epsr)))
H = 4.*np.pi/dphidz
layer_dz = 2.
zs = np.arange(1,H,layer_dz)
thetas = np.zeros(len(zs))
psis = .01+np.zeros_like(zs)
psis[zs>200] = .6
dzs = layer_dz*np.ones(len(zs))
chis = np.tile(chis,(len(zs),1))
em.solve(zs,dzs,thetas,psis,chis)

dat = load_quadpol_fujita(em)
dat.rotational_transform(n_thetas=401)
dat.find_cpe(Wn=3e7,rad_start=0.1*np.pi,rad_end=.6*np.pi)
dat.coherence2d(.15,layer_dz+.1)

In [ ]:
plt.figure(figsize=(12,4))
ax1 = plt.subplot(131,projection='3d')
ax1.plot_wireframe(y, z, x, color="k", alpha=0.025)
ax2 = plt.subplot(132,projection='3d')
ax2.plot_wireframe(y, z, x, color="k", alpha=0.025)
ax3 = plt.subplot(133,projection='3d')
ax3.plot_wireframe(y, z, x, color="k", alpha=0.025)

S1, S2, S3 = get_poincare_sphere(dat5)
i = 70
ax1.plot(S2[:150,i],-S1[:150,i],S3[:150,i],lw=2,c=cm.twilight_shifted(i/200))
ax1.plot(S2[150:,i],-S1[150:,i],S3[150:,i],lw=2,c='grey')
ax1.plot(S2[150,i],-S1[150,i],S3[150,i],'.',ms=10,mew=2,mfc='w',c='grey')
i = 110
ax2.plot(S2[:150,i],-S1[:150,i],S3[:150,i],lw=2,c=cm.twilight_shifted(i/200))
ax2.plot(S2[150:,i],-S1[150:,i],S3[150:,i],lw=2,c='grey')
ax2.plot(S2[150,i],-S1[150,i],S3[150,i],'.',ms=10,mew=2,mfc='w',c='grey')
i = 130
ax3.plot(S2[:150,i],-S1[:150,i],S3[:150,i],lw=2,c=cm.twilight_shifted(i/200))
ax3.plot(S2[150:,i],-S1[150:,i],S3[150:,i],lw=2,c='grey')
ax3.plot(S2[150,i],-S1[150,i],S3[150,i],'.',ms=10,mew=2,mfc='w',c='grey')

ax1.axis('equal')
ax1.axis('off')
ax2.axis('equal')
ax2.axis('off')
ax3.axis('equal')
ax3.axis('off')

print(i/200+.2)

plt.savefig('Sphere_Rotated_Azimuth.pdf')